In [1]:
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
import ase.io
from ase.optimize import BFGS

from ocdata.core import Adsorbate, AdsorbateSlabConfig, Bulk, Slab
import os
from glob import glob
import pandas as pd
from ocdata.utils import DetectTrajAnomaly
from ocdata.utils.vasp import write_vasp_input_files

# Optional - see below
import numpy as np
# from dscribe.descriptors import SOAP
from scipy.spatial.distance import pdist, squareform
from x3dase.visualize import view_x3d_n

# Enumerate the adsorbate-slab configurations to run relaxations on

Be sure to set the path to the bulk and adsorbate pickle files in `ocdata/configs/paths.py` or pass the paths as an argument. The database pickles can be found in `ocdata/databases/pkls`. AdsorbML incorporates random placement, which is especially useful for more complicated adsorbates which may have many degrees of freedom. I have opted sample a few random placements and a few heuristic. Here I am using *CO on copper (1,1,1) as an example.

In [18]:
bulk_src_id = "mp-79"
adsorbate_smiles = "*CHO"

bulk = Bulk(bulk_src_id_from_db = bulk_src_id)
adsorbate = Adsorbate(adsorbate_smiles_from_db=adsorbate_smiles)
slabs = Slab.from_bulk_get_specific_millers(bulk = bulk, specific_millers=(2,1,1))

# There may be multiple slabs with this miller index.
# For demonstrative purposes we will take the first entry.
slab = slabs[1]

In [3]:
slabs

[Slab: (Zn72, (2, 1, 1), 0.0, True),
 Slab: (Zn72, (2, 1, 1), 0.16666666666666674, True)]

In [19]:
# Perform heuristic placements
heuristic_adslabs = AdsorbateSlabConfig(slabs[0], adsorbate, mode="heuristic")

# Perform random placements
# (for AdsorbML we use `num_sites = 100` but we will use 4 for brevity here)
random_adslabs = AdsorbateSlabConfig(slabs[0], adsorbate, mode="random_site_heuristic_placement", num_sites = 50)

In [4]:
adslabs =  random_adslabs.sites

# Run ML relaxations:

There are 2 options for how to do this.
 1. Using `OCPCalculator` as the calculator within the ASE framework
 2. By writing objects to lmdb and relaxing them using `main.py` in the ocp repo
 
(1) is really only adequate for small stuff and it is what I will show here, but if you plan to run many relaxations, you should definitely use (2). More details about writing lmdbs has been provided [here](https://github.com/Open-Catalyst-Project/ocp/blob/main/tutorials/lmdb_dataset_creation.ipynb) - follow the IS2RS/IS2RE instructions. And more information about running relaxations once the lmdb has been written is [here](https://github.com/Open-Catalyst-Project/ocp/blob/main/TRAIN.md#initial-structure-to-relaxed-structure-is2rs).

You need to provide the calculator with a path to a model checkpoint file. That can be downloaded [here](../core/model_checkpoints)


In [20]:
checkpoint_path = "/home/jovyan/checkpoints/eq2_31M_ec4_allmd.pt"
os.makedirs(f"data/{bulk}_{adsorbate}", exist_ok=True)

# Define the calculator
calc = OCPCalculator(checkpoint_path=checkpoint_path, cpu = False) # if you dont have a gpu, change to `cpu=True` to run on cpu

adslabs = [*heuristic_adslabs.atoms_list, *random_adslabs.atoms_list]
# Set up the calculator
for idx, adslab in enumerate(adslabs):
    adslab.calc = calc
    opt = BFGS(adslab, trajectory=f"data/{bulk}_{adsorbate}/{idx}.traj")
    opt.run(fmax=0.05, steps=100) # For the AdsorbML results we used fmax = 0.02 and steps = 300, but we will use less strict values for brevity.

      Step     Time          Energy         fmax
BFGS:    0 15:06:38        3.599609        9.0761
BFGS:    1 15:06:38        2.763672        2.2793
BFGS:    2 15:06:38        2.566406        1.3214
BFGS:    3 15:06:38        2.246094        1.6093
BFGS:    4 15:06:38        2.054688        1.9848
BFGS:    5 15:06:38        1.826172        1.8646
BFGS:    6 15:06:38        1.632812        1.8983
BFGS:    7 15:06:38        1.435547        1.6057
BFGS:    8 15:06:38        1.279297        1.0426
BFGS:    9 15:06:38        1.172852        1.0528
BFGS:   10 15:06:39        1.077148        1.1439
BFGS:   11 15:06:39        0.999512        0.8943
BFGS:   12 15:06:39        0.934082        0.9396
BFGS:   13 15:06:39        0.881348        0.7552
BFGS:   14 15:06:39        0.823730        0.7613
BFGS:   15 15:06:39        0.773926        0.6887
BFGS:   16 15:06:39        0.751465        0.5834
BFGS:   17 15:06:39        0.710449        0.6581
BFGS:   18 15:06:39        0.660645        0.7085
B

BFGS:   61 15:06:51        0.008713        0.2183
BFGS:   62 15:06:51        0.006763        0.3165
BFGS:   63 15:06:51        0.002855        0.2309
BFGS:   64 15:06:51        0.001390        0.1920
BFGS:   65 15:06:51       -0.003492        0.2296
BFGS:   66 15:06:51       -0.005932        0.2245
BFGS:   67 15:06:51       -0.013260        0.2187
BFGS:   68 15:06:51       -0.022537        0.1464
BFGS:   69 15:06:51       -0.028885        0.0947
BFGS:   70 15:06:51       -0.030838        0.0706
BFGS:   71 15:06:51       -0.032288        0.0775
BFGS:   72 15:06:52       -0.037170        0.0862
BFGS:   73 15:06:52       -0.040588        0.0876
BFGS:   74 15:06:52       -0.039124        0.0873
BFGS:   75 15:06:52       -0.039124        0.1062
BFGS:   76 15:06:52       -0.040588        0.0819
BFGS:   77 15:06:52       -0.048889        0.0557
BFGS:   78 15:06:52       -0.048889        0.0517
BFGS:   79 15:06:52       -0.049866        0.0610
BFGS:   80 15:06:52       -0.052795        0.0689


BFGS:   40 15:07:04       -0.092834        0.4871
BFGS:   41 15:07:04       -0.106506        0.6343
BFGS:   42 15:07:04       -0.122620        0.8466
BFGS:   43 15:07:04       -0.136841        0.5813
BFGS:   44 15:07:04       -0.149048        0.2821
BFGS:   45 15:07:04       -0.151978        0.1804
BFGS:   46 15:07:04       -0.158325        0.2164
BFGS:   47 15:07:04       -0.174438        0.3973
BFGS:   48 15:07:04       -0.186157        0.4768
BFGS:   49 15:07:04       -0.194458        0.4689
BFGS:   50 15:07:04       -0.200317        0.2585
BFGS:   51 15:07:04       -0.208130        0.2735
BFGS:   52 15:07:04       -0.211548        0.2975
BFGS:   53 15:07:05       -0.216431        0.3267
BFGS:   54 15:07:05       -0.221313        0.2637
BFGS:   55 15:07:05       -0.225708        0.2361
BFGS:   56 15:07:05       -0.233032        0.2877
BFGS:   57 15:07:05       -0.236938        0.2264
BFGS:   58 15:07:05       -0.242798        0.1514
BFGS:   59 15:07:05       -0.246216        0.1113


BFGS:   48 15:07:16       -0.212524        0.2733
BFGS:   49 15:07:17       -0.225220        0.3318
BFGS:   50 15:07:17       -0.233032        0.2576
BFGS:   51 15:07:17       -0.238892        0.1603
BFGS:   52 15:07:17       -0.243774        0.1369
BFGS:   53 15:07:17       -0.246216        0.2472
BFGS:   54 15:07:17       -0.248169        0.3606
BFGS:   55 15:07:17       -0.255859        0.3454
BFGS:   56 15:07:17       -0.261475        0.1672
BFGS:   57 15:07:17       -0.261230        0.1469
BFGS:   58 15:07:17       -0.264648        0.1851
BFGS:   59 15:07:17       -0.267578        0.2882
BFGS:   60 15:07:17       -0.268311        0.2998
BFGS:   61 15:07:17       -0.269775        0.1932
BFGS:   62 15:07:18       -0.270996        0.1851
BFGS:   63 15:07:18       -0.272949        0.1386
BFGS:   64 15:07:18       -0.274902        0.1925
BFGS:   65 15:07:18       -0.283447        0.2274
BFGS:   66 15:07:18       -0.290039        0.1463
BFGS:   67 15:07:18       -0.293701        0.0859


BFGS:   58 15:07:29       -0.038147        0.2145
BFGS:   59 15:07:29       -0.044006        0.2768
BFGS:   60 15:07:29       -0.047913        0.2469
BFGS:   61 15:07:29       -0.048889        0.1209
BFGS:   62 15:07:30       -0.052307        0.0642
BFGS:   63 15:07:30       -0.057678        0.0813
BFGS:   64 15:07:30       -0.060608        0.1306
BFGS:   65 15:07:30       -0.062073        0.1410
BFGS:   66 15:07:30       -0.064026        0.1194
BFGS:   67 15:07:30       -0.062073        0.0477
      Step     Time          Energy         fmax
BFGS:    0 15:07:30        2.566406        4.6030
BFGS:    1 15:07:30        2.408203        2.2483
BFGS:    2 15:07:30        2.339844        1.3881
BFGS:    3 15:07:30        2.142578        2.0982
BFGS:    4 15:07:30        1.994141        2.1130
BFGS:    5 15:07:30        1.703125        1.6660
BFGS:    6 15:07:30        1.574219        1.4631
BFGS:    7 15:07:31        1.416016        0.7806
BFGS:    8 15:07:31        1.380859        0.7315
B

...

BFGS:   36 15:14:42        0.272949        0.3443
BFGS:   37 15:14:42        0.253418        0.5342
BFGS:   38 15:14:42        0.235718        0.4931
BFGS:   39 15:14:42        0.205933        0.4149
BFGS:   40 15:14:42        0.203979        0.3339
BFGS:   41 15:14:42        0.198120        0.2688
BFGS:   42 15:14:42        0.186890        0.3740
BFGS:   43 15:14:43        0.176636        0.4502
BFGS:   44 15:14:43        0.170776        0.3264
BFGS:   45 15:14:43        0.163940        0.1671
BFGS:   46 15:14:43        0.140991        0.1707
BFGS:   47 15:14:43        0.137085        0.2990
BFGS:   48 15:14:43        0.129761        0.4125
BFGS:   49 15:14:43        0.110779        0.3628
BFGS:   50 15:14:43        0.101013        0.2325
BFGS:   51 15:14:43        0.097595        0.2488
BFGS:   52 15:14:43        0.091736        0.2398
BFGS:   53 15:14:43        0.080505        0.3737
BFGS:   54 15:14:43        0.072693        0.4316
BFGS:   55 15:14:43        0.061462        0.3143


BFGS:    5 15:14:54        0.441895        0.6836
BFGS:    6 15:14:54        0.407715        0.3937
BFGS:    7 15:14:55        0.381348        0.6353
BFGS:    8 15:14:55        0.345215        0.8538
BFGS:    9 15:14:55        0.312988        0.6013
BFGS:   10 15:14:55        0.297363        0.3093
BFGS:   11 15:14:55        0.283691        0.3573
BFGS:   12 15:14:55        0.259277        0.7571
BFGS:   13 15:14:55        0.224487        1.0407
BFGS:   14 15:14:55        0.189331        0.9753
BFGS:   15 15:14:55        0.161011        0.4939
BFGS:   16 15:14:55        0.155151        0.3404
BFGS:   17 15:14:55        0.142456        0.3891
BFGS:   18 15:14:56        0.126831        0.5317
BFGS:   19 15:14:56        0.116638        0.4163
BFGS:   20 15:14:56        0.105408        0.3087
BFGS:   21 15:14:56        0.096619        0.3895
BFGS:   22 15:14:56        0.086365        0.4376
BFGS:   23 15:14:56        0.067322        0.5049
BFGS:   24 15:14:56        0.051208        0.3644


BFGS:   67 15:15:07        0.034607        0.0961
BFGS:   68 15:15:07        0.032166        0.1193
BFGS:   69 15:15:07        0.033142        0.1572
BFGS:   70 15:15:07        0.029221        0.1523
BFGS:   71 15:15:07        0.024826        0.1007
BFGS:   72 15:15:07        0.022873        0.0956
BFGS:   73 15:15:07        0.018478        0.0967
BFGS:   74 15:15:07        0.018478        0.0822
BFGS:   75 15:15:08        0.016525        0.0577
BFGS:   76 15:15:08        0.011642        0.0596
BFGS:   77 15:15:08        0.008713        0.0581
BFGS:   78 15:15:08        0.007252        0.0453
      Step     Time          Energy         fmax
BFGS:    0 15:15:08        2.267578        2.9567
BFGS:    1 15:15:08        2.126953        2.0097
BFGS:    2 15:15:08        1.994141        1.6002
BFGS:    3 15:15:08        1.757812        1.5891
BFGS:    4 15:15:08        1.619141        1.6560
BFGS:    5 15:15:08        1.482422        1.6554
BFGS:    6 15:15:08        1.291016        1.3284
B

BFGS:   64 15:15:19        0.076599        0.0811
BFGS:   65 15:15:19        0.070251        0.0766
BFGS:   66 15:15:19        0.056091        0.0956
BFGS:   67 15:15:19        0.045837        0.0884
BFGS:   68 15:15:19        0.040955        0.0811
BFGS:   69 15:15:20        0.043396        0.0736
BFGS:   70 15:15:20        0.038513        0.0811
BFGS:   71 15:15:20        0.034607        0.1432
BFGS:   72 15:15:20        0.029709        0.1751
BFGS:   73 15:15:20        0.021896        0.1269
BFGS:   74 15:15:20        0.016525        0.1026
BFGS:   75 15:15:20        0.015060        0.0848
BFGS:   76 15:15:20        0.011642        0.0980
BFGS:   77 15:15:20        0.007740        0.1723
BFGS:   78 15:15:20        0.013596        0.1859
BFGS:   79 15:15:20        0.015060        0.1767
BFGS:   80 15:15:20        0.019455        0.1552
BFGS:   81 15:15:20        0.015549        0.1140
BFGS:   82 15:15:21        0.010178        0.1401
BFGS:   83 15:15:21        0.010666        0.1646


BFGS:   24 15:15:31        0.763184        0.9718
BFGS:   25 15:15:32        0.737793        0.5623
BFGS:   26 15:15:32        0.712402        0.3790
BFGS:   27 15:15:32        0.695801        0.3430
BFGS:   28 15:15:32        0.672363        0.4484
BFGS:   29 15:15:32        0.660645        0.3669
BFGS:   30 15:15:32        0.651855        0.6662
BFGS:   31 15:15:32        0.643066        0.5058
BFGS:   32 15:15:32        0.627441        0.2910
BFGS:   33 15:15:32        0.618652        0.2115
BFGS:   34 15:15:32        0.595215        0.1401
BFGS:   35 15:15:32        0.561035        0.3469
BFGS:   36 15:15:32        0.539551        0.5802
BFGS:   37 15:15:32        0.514160        0.3815
BFGS:   38 15:15:33        0.497559        0.3503
BFGS:   39 15:15:33        0.490723        0.2160
BFGS:   40 15:15:33        0.464355        0.3724
BFGS:   41 15:15:33        0.426270        0.5730
BFGS:   42 15:15:33        0.384277        0.3302
BFGS:   43 15:15:33        0.374512        0.2703


# Parse the trajectories and post-process

As a post-processing step we check to see if:
1. the adsorbate desorbed
2. the adsorbate disassociated
3. the adsorbate intercalated
4. the surface has changed

We check these because they effect our referencing scheme and may result in erroneous energies. For (4), the relaxed surface should really be supplied as well. It will be necessary when correcting the SP / RX energies later. Since we don't have it here, we will ommit supplying it, and the detector will instead compare the initial and final slab from the adsorbate-slab relaxation trajectory. If a relaxed slab is provided, the detector will compare it and the slab after the adsorbate-slab relaxation. The latter is more correct! Note: for the results in the AdsorbML paper, we did not check if the adsorbate was intercalated (`is_adsorbate_intercalated()`) because it is a new addition.

In [21]:
# Iterate over trajs to extract results
results = []
for file in glob(f"data/{bulk}_{adsorbate}/*.traj"):
    rx_id = file.split("/")[-1].split(".")[0]
    traj = ase.io.read(file, ":")
    
    # Check to see if the trajectory is anomolous
    initial_atoms = traj[0]
    final_atoms = traj[-1]
    atom_tags = initial_atoms.get_tags()
    detector = DetectTrajAnomaly(initial_atoms, final_atoms, atom_tags)
    anom = (
        detector.is_adsorbate_dissociated()
        or detector.is_adsorbate_desorbed()
        or detector.has_surface_changed()
        or detector.is_adsorbate_intercalated()
    )
    rx_energy = traj[-1].get_potential_energy()
    results.append({"relaxation_idx": rx_id, "relaxed_atoms": traj[-1],
                    "relaxed_energy_ml": rx_energy, "anomolous": anom})

In [22]:
df = pd.DataFrame(results)
df.relaxed_energy_ml.min()

-0.41015625

In [24]:
#scrap anomalies
df = df[~df.anomolous].copy().reset_index()

In [25]:
df.relaxed_energy_ml.min()

-0.365478515625

In [26]:
df.sort_values(by = "relaxed_energy_ml", inplace = True)

In [27]:
atoms = df.relaxed_atoms.tolist()

In [29]:
view_x3d_n(atoms[0].repeat((3,3,1)))

# (Optional) Deduplicate structures
We may have enumerated very similar structures or structures may have relaxed to the same configuration. For this reason, it is advantageous to cull systems if they are very similar. This results in marginal improvements in the recall metrics we calculated for AdsorbML, so it wasnt implemented there. It is, however, a good way to prevent wasteful VASP calculations. You can also imagine that if we would have enumerated 1000 configs per slab adsorbate combo rather than 100 for AdsorbML, it is more likely that having redundant systems would reduce performance, so its a good thing to keep in mind. This may be done by eye for a small number of systems, but with many systems it is easier to use an automated approach. Here is an example of one such approach, which uses a SOAP descriptor to find similar systems.

In [ ]:
# Extract the configs and their energies
def deduplicate(configs_for_deduplication: list,
                adsorbate_binding_index: int,
                cosine_similarity = 1e-3,
               ):
    """
    A function that may be used to deduplicate similar structures.
    Among duplicate entries, the one with the lowest energy will be kept.
    
    Args:
        configs_for_deduplication: a list of ML relaxed adsorbate-
            surface configurations.
        cosine_similarity: The cosine simularity value above which,
            configurations are considered duplicate.
            
    Returns:
        (list): the indices of configs which should be kept as non-duplicate
    """
    
    energies_for_deduplication = np.array([atoms.get_potential_energy() for atoms in configs_for_deduplication])
    # Instantiate the soap descriptor
    soap = SOAP(
        species=np.unique(configs_for_deduplication[0].get_chemical_symbols()),
        r_cut = 2.0,
        n_max=6,
        l_max=3,
        periodic=True,
    )
    #Figure out which index cooresponds to 
    ads_len = list(configs_for_deduplication[0].get_tags()).count(2)
    position_idx = -1*(ads_len-adsorbate_binding_index)
    # Iterate over the systems to get the SOAP vectors
    soap_desc = []
    for config in configs_for_deduplication:
        soap_ex = soap.create(config, centers=[position_idx])
        soap_desc.extend(soap_ex)

    soap_descs = np.vstack(soap_desc)

    #Use euclidean distance to assess similarity
    distance = squareform(pdist(soap_descs, metric="cosine"))

    bool_matrix = np.where(distance <= cosine_similarity, 1, 0)
    # For configs that are found to be similar, just keep the lowest energy one
    idxs_to_keep = []
    pass_idxs = []
    for idx, row in enumerate(bool_matrix):
        if idx in pass_idxs:
            continue
            
        elif sum(row) == 1:
            idxs_to_keep.append(idx)
        else:
            same_idxs = [row_idx for row_idx, val in enumerate(row) if val == 1]
            pass_idxs.extend(same_idxs)
            # Pick the one with the lowest energy by ML
            min_e = min(energies_for_deduplication[same_idxs])
            idxs_to_keep.append(list(energies_for_deduplication).index(min_e))
    return idxs_to_keep

In [ ]:
configs_for_deduplication =  df.relaxed_atoms.tolist()
idxs_to_keep = deduplicate(configs_for_deduplication, adsorbate.binding_indices[0])

In [ ]:
# Flip through your configurations to check them out (and make sure deduplication looks good)
print(idxs_to_keep)
view_x3d_n(configs_for_deduplication[2].repeat((2,2,1)))

In [ ]:
df = df.iloc[idxs_to_keep]

In [ ]:
low_e_values = np.round(df.sort_values(by = "relaxed_energy_ml").relaxed_energy_ml.tolist()[0:5],3)
print(f"The lowest 5 energies are: {low_e_values}")
df

# Write VASP input files

This assumes you have access to VASP pseudopotentials. The default VASP flags (which are equivalent to those used to make OC20) are located in `ocdata.utils.vasp`. Alternatively, you may pass your own vasp flags to the `write_vasp_input_files` function as `vasp_flags`

In [ ]:
# Grab the 5 systems with the lowest energy
configs_for_dft = df.sort_values(by = "relaxed_energy_ml").relaxed_atoms.tolist()[0:5]
config_idxs = df.sort_values(by = "relaxed_energy_ml").relaxation_idx.tolist()[0:5]

# Write the inputs
for idx, config in enumerate(configs_for_dft):
    os.mkdir(f"data/{config_idxs[idx]}")
    write_vasp_input_files(config, outdir = f"data/{config_idxs[idx]}/")